In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso,Ridge,BayesianRidge,ElasticNet,HuberRegressor,LinearRegression,LogisticRegression,SGDRegressor
from sklearn.metrics import mean_squared_error
import joblib

In [2]:
# model load
loaded_model = joblib.load('./model.pkl')

In [3]:
# load data

'''
brand model year transmission mileage fuelType tax mpg engineSize
'''
json_data = '''
[{
    "brand" : "bmw",
    "model" : "z3",
    "year" : "2020",
    "transmission" : "Automatic",
    "mileage" : "61123",
    "fuelType" : "Petrol",
    "tax" : "160.0",
    "mpg" : "62",
    "engineSize" : "2.0"
}]
'''

In [4]:
# convert to dataframe
import json
from pandas import json_normalize
a_json = json.loads(json_data)
df = json_normalize(a_json)
df

,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,bmw,z3,2020,Automatic,61123,Petrol,160.0,62,2.0


In [5]:
# data dtype parse
'''
year, mileage - int64
tax, mpg, engineSize - float64
'''
df = df.astype({'year' : 'int64'})
df = df.astype({'mileage' : 'int64'})
df = df.astype({'tax' : 'float64'})
df = df.astype({'mpg' : 'float64'})
df = df.astype({'engineSize' : 'float64'})

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         1 non-null      object 
 1   model         1 non-null      object 
 2   year          1 non-null      int64  
 3   transmission  1 non-null      object 
 4   mileage       1 non-null      int64  
 5   fuelType      1 non-null      object 
 6   tax           1 non-null      float64
 7   mpg           1 non-null      float64
 8   engineSize    1 non-null      float64
dtypes: float64(3), int64(2), object(4)
memory usage: 200.0+ bytes


In [7]:
'''데이터 전처리'''
# brand 변경
df.replace({'brand' : { 'audi' : 1, 'bmw' : 2, 'merc' : 3, 'vw': 4, 'toyota': 5, 'ford': 6, 'hyundi': 7, 'skoda': 8, 'vauxhall':9 }},inplace=True)
# transmission 변경 이거 autmatic 어쨌음 .. ?
df['transmission_Manual'] = 0
df['transmission_Semi-Auto'] = 1
df.drop(['transmission'], axis = 1, inplace=True)
# fuelType 변경
df['fuelType_Petrol'] = 1
df['fuelType_Hybrid'] = 0
df['fuelType_Other'] = 0
df.drop(['fuelType'], axis = 1, inplace=True)
# model, year 제거
df.drop(['model', 'year'], axis = 1, inplace=True)
# scaler 사용
#from sklearn.preprocessing import StandardScaler # Converts Columnar Data into Standard Normal Distribution
#scaler=StandardScaler()
#scaler.fit(df)
#scaled_data = scaler.transform(df)
# pca 사용
#from sklearn.decomposition import PCA # Reduce Dimensions by Principal Component Analysis To Compensate for Variables with High VIF
#pca = PCA(n_components = 5)
#pca.fit(df)
#x_pca = pca.transform(df)
#x_pca


df

,brand,mileage,tax,mpg,engineSize,transmission_Manual,transmission_Semi-Auto,fuelType_Petrol,fuelType_Hybrid,fuelType_Other
0,2,61123,160.0,62.0,2.0,0,1,1,0,0


In [13]:
from sklearn.preprocessing import StandardScaler # Converts Columnar Data into Standard Normal Distribution
temp = np.array([[2, 61123, 160.0, 62.0, 2.0, 0, 1, 1, 0, 0]])
scaler=StandardScaler()
scaler.fit(temp)
scaled_data = scaler.transform(temp)

In [17]:
from sklearn.decomposition import PCA # Reduce Dimensions by Principal Component Analysis To Compensate for Variables with High VIF
pca = PCA(n_components = 1)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)
x_pca

/usr/local/lib/python3.7/site-packages/sklearn/decomposition/_pca.py:461: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)


array([[0.]])

In [18]:
pred = loaded_model.predict(x_pca)

ValueError: Number of features of the model must match the input. Model n_features is 5 and input n_features is 1 